In [183]:
import sqlite3
import pandas as pd
import wbdata as wb

In [184]:
# Load indicators into DataFrames
# GDP per capita (constant 2015 US$)
gdppc_df = wb.get_dataframe(
    {'NY.GDP.PCAP.KD': 'gdp_per_capita'},
    parse_dates=True
).reset_index()

# Employment to population ratio
epr_df = wb.get_dataframe(
    {'SL.EMP.TOTL.SP.ZS': 'employment_population_ratio'},
    parse_dates=True
).reset_index()

# GDP growth (annual %)
gdp_growth_df = wb.get_dataframe(
    {'NY.GDP.MKTP.KD.ZG': 'gdp_growth'},
    parse_dates=True
).reset_index()

# Population growth (annual %)
pop_growth_df = wb.get_dataframe(
    {'SP.POP.GROW': 'population_growth'},
    parse_dates=True
).reset_index()

In [185]:
# Connect to the SQLite database
conn = sqlite3.connect('wdi_data.db')

# Save indicator DataFrames to SQLite as tables
gdppc_df.to_sql('gdp_per_capita', conn, if_exists='replace', index=False)
epr_df.to_sql('employment_population_ratio', conn, if_exists='replace', index=False)
gdp_growth_df.to_sql('gdp_growth', conn, if_exists='replace', index=False)
pop_growth_df.to_sql('population_growth', conn, if_exists='replace', index=False)

# Save indicator DataFrames to CSV
gdppc_df.to_csv("gdp_per_capita.csv", index=False)
epr_df.to_csv("employment_population_ratio.csv", index=False)
gdp_growth_df.to_csv("gdp_growth.csv", index=False)
pop_growth_df.to_csv("population_growth.csv", index=False)

In [186]:
# Create a temporary view of four indicators for Asian countries
conn.execute('''
CREATE TEMP VIEW asian_countries AS
SELECT 
    g.country,
    CAST(STRFTIME('%Y', g.date) AS INTEGER) AS year,
    g.gdp_per_capita,
    e.employment_population_ratio,
    gr.gdp_growth,
    pr.population_growth
FROM gdp_per_capita g
LEFT JOIN employment_population_ratio e
    ON g.country = e.country AND g.date = e.date
LEFT JOIN gdp_growth gr
    ON g.country = gr.country AND g.date = gr.date
LEFT JOIN population_growth pr
    ON g.country = pr.country AND g.date = pr.date
WHERE g.country IN (
    -- South Asia
    'Afghanistan', 'Bangladesh', 'Bhutan', 'India',
    'Maldives', 'Nepal', 'Pakistan', 'Sri Lanka',

    -- East Asia
    'China', 'Hong Kong SAR, China', 'Japan', 'Korea, Dem. People''s Rep.', 
    'Korea, Rep.', 'Macao SAR, China', 'Mongolia',
             
    -- Southeast Asia
    'Brunei Darussalam', 'Cambodia', 'Indonesia', 'Lao PDR', 
    'Malaysia', 'Myanmar', 'Philippines', 'Singapore',
    'Thailand', 'Timor-Leste', 'Viet Nam',

    -- Central Asia
    'Armenia', 'Azerbaijan', 'Georgia', 'Kazakhstan', 'Kyrgyz Republic', 
    'Tajikistan', 'Turkmenistan', 'Uzbekistan',

    -- Middle East & North Africa (Western Asia)
    'Bahrain', 'Cyprus', 'Egypt, Arab Rep.', 'Iran, Islamic Rep.', 'Iraq', 'Israel',
    'Jordan', 'Kuwait', 'Lebanon', 'Oman', 'Qatar', 'Saudi Arabia', 'Syrian Arab Republic', 
    'Turkiye', 'United Arab Emirates', 'West Bank and Gaza', 'Yemen, Rep.'
);
''')

In [187]:
# Compute summary statistics for each country in the view
desc_stats_df = pd.read_sql_query('''
SELECT 
    country,
    COUNT(*) AS n_years,
    AVG(gdp_per_capita) AS avg_gdp_per_capita,
    MIN(gdp_per_capita) AS min_gdp_per_capita,
    MAX(gdp_per_capita) AS max_gdp_per_capita,
    AVG(employment_population_ratio) AS avg_employment_ratio,
    AVG(gdp_growth) AS avg_gdp_growth,
    AVG(population_growth) AS avg_population_growth
FROM asian_countries
GROUP BY country
ORDER BY country;
''', conn)

desc_stats_df

,country,n_years,avg_gdp_per_capita,min_gdp_per_capita,max_gdp_per_capita,avg_employment_ratio,avg_gdp_growth,avg_population_growth
0,Afghanistan,65,456.122012,277.118051,580.603833,41.497676,4.602173,2.418241
1,Armenia,65,2507.861035,792.567859,5151.012731,54.121147,3.660196,0.750806
2,Azerbaijan,65,3539.166619,1102.497890,5651.345639,63.766529,3.916412,1.521108
3,Bahrain,65,22079.575222,17091.290302,25407.969551,68.263735,4.617434,3.578547
4,Bangladesh,65,700.522162,373.260374,1885.377336,56.085118,4.406776,1.899123
5,Bhutan,65,1417.675578,370.056039,3467.385989,65.957294,6.126208,1.992739
6,Brunei Darussalam,65,44527.469870,28549.175528,96517.133293,62.565265,0.794002,2.615344
7,Cambodia,65,1007.803790,496.770551,2083.593018,80.438412,4.560490,1.844813
8,China,65,2879.983541,163.907044,12175.196115,68.925618,7.947512,1.188815
9,Cyprus,65,20082.208536,6173.291992,32341.302734,59.202647,4.941771,1.317422


#### Yearly Trends in Economic Development Indicators (2000–2022)

In [188]:
# Average GDP per Capita Over Time
pd.read_sql_query('''
SELECT 
    year,
    AVG(gdp_per_capita) AS avg_gdp_per_capita
FROM asian_countries
WHERE year BETWEEN 2000 AND 2022
GROUP BY year
ORDER BY year
''', conn)

,year,avg_gdp_per_capita
0,2000,10393.108037
1,2001,10306.459501
2,2002,10403.438332
3,2003,10735.568288
4,2004,11459.399661
5,2005,11754.745019
6,2006,12255.707301
7,2007,12450.514358
8,2008,12419.280644
9,2009,12082.596983


In [189]:
# Average Employment to Population Ratio Over Time
pd.read_sql_query('''
SELECT 
    year,
    AVG(employment_population_ratio) AS avg_employment_ratio
FROM asian_countries
WHERE year BETWEEN 2000 AND 2022
GROUP BY year
ORDER BY year
''', conn)

,year,avg_employment_ratio
0,2000,57.047608
1,2001,56.779804
2,2002,56.401471
3,2003,56.330020
4,2004,56.263706
5,2005,56.275667
6,2006,56.336824
7,2007,56.510745
8,2008,56.236020
9,2009,56.138137


In [190]:
# Average GDP Growth Over Time
pd.read_sql_query('''
SELECT 
    year,
    AVG(gdp_growth) AS avg_gdp_growth
FROM asian_countries
WHERE year BETWEEN 2000 AND 2022
GROUP BY year
ORDER BY year
''', conn)

,year,avg_gdp_growth
0,2000,7.002528
1,2001,3.474163
2,2002,4.573037
3,2003,5.815277
4,2004,8.598225
5,2005,6.761817
6,2006,7.812865
7,2007,7.993075
8,2008,5.749328
9,2009,2.898034


#### Leading Countries by Economic Development Indicator (2022)

In [191]:
# Top 10 Countries by GDP per Capita in 2022
pd.read_sql_query('''
SELECT 
    country,
    gdp_per_capita
FROM asian_countries
WHERE year = '2022'
ORDER BY gdp_per_capita DESC
LIMIT 10
''', conn)

,country,gdp_per_capita
0,Singapore,67948.892827
1,Qatar,64315.626367
2,"Hong Kong SAR, China",43281.795509
3,Israel,42710.401759
4,United Arab Emirates,42687.720718
5,Japan,36202.639275
6,"Macao SAR, China",34121.950823
7,"Korea, Rep.",33690.378879
8,Cyprus,32015.351562
9,Brunei Darussalam,28549.175528


In [192]:
# Top 10 Countries by Employment to Population Ratio in 2022
pd.read_sql_query('''
SELECT 
    country,
    employment_population_ratio
FROM asian_countries
WHERE year = '2022'
ORDER BY employment_population_ratio DESC
LIMIT 10
''', conn)

,country,employment_population_ratio
0,Qatar,87.461
1,Cambodia,79.833
2,"Korea, Dem. People's Rep.",79.808
3,United Arab Emirates,75.023
4,Viet Nam,73.347
5,Kuwait,72.686
6,Bahrain,69.477
7,Kazakhstan,68.337
8,Singapore,67.230
9,Thailand,66.549


In [193]:
# Top 10 Countries by GDP Growth in 2022
pd.read_sql_query('''
SELECT 
    country,
    gdp_growth
FROM asian_countries
WHERE year = '2022'
ORDER BY gdp_growth DESC
LIMIT 10
''', conn)

,country,gdp_growth
0,Maldives,13.826090
1,Armenia,12.600000
2,Georgia,10.958532
3,Oman,9.577106
4,Kyrgyz Republic,8.967505
5,Malaysia,8.861822
6,Viet Nam,8.123514
7,Tajikistan,8.000000
8,Iraq,7.639811
9,Philippines,7.580982


In [194]:
# Load data from the view into a DataFrame
wdi_df = pd.read_sql_query("SELECT * FROM asian_countries", conn)

# Save merged data from asian_countries view
wdi_df.to_csv("asian_countries.csv", index=False)

In [195]:
wdi_df

,country,year,gdp_per_capita,employment_population_ratio,gdp_growth,population_growth
0,Afghanistan,2024,NaN,32.511,NaN,NaN
1,Afghanistan,2023,379.707497,32.402,2.710887,2.135594
2,Afghanistan,2022,377.665627,32.333,-6.240172,1.435704
3,Afghanistan,2021,408.625855,36.022,-20.738839,2.356098
4,Afghanistan,2020,527.834554,36.710,-2.351101,3.153609
...,...,...,...,...,...,...
3310,"Yemen, Rep.",1964,NaN,NaN,NaN,2.289077
3311,"Yemen, Rep.",1963,NaN,NaN,NaN,2.205000
3312,"Yemen, Rep.",1962,NaN,NaN,NaN,2.220673
3313,"Yemen, Rep.",1961,NaN,NaN,NaN,2.197731
